In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import collections

In [2]:
# Copyright (c) 2017-2019 The University of Manchester
#
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with this program.  If not, see <http://www.gnu.org/licenses/>.
from collections import OrderedDict
from enum import Enum
import struct

from spinn_front_end_common.abstract_models import AbstractProvidesNKeysForPartition
from spinn_utilities.overrides import overrides
from pacman.executor.injection_decorator import inject_items
from pacman.model.graphs.machine import MachineVertex
from pacman.model.resources import ResourceContainer, VariableSDRAM
from pacman.utilities.utility_calls import is_single
from spinn_front_end_common.utilities.constants import (
    SYSTEM_BYTES_REQUIREMENT, BYTES_PER_WORD)
from spinn_front_end_common.utilities.exceptions import ConfigurationException
from spinn_front_end_common.utilities.helpful_functions import (
    locate_memory_region_for_placement)
from spinn_front_end_common.abstract_models.impl import (
    MachineDataSpecableVertex)
from spinn_front_end_common.interface.buffer_management.buffer_models import (
    AbstractReceiveBuffersToHost)
from spinn_front_end_common.interface.buffer_management import (
    recording_utilities)
from spinnaker_graph_front_end.utilities import SimulatorVertex
from spinnaker_graph_front_end.utilities.data_utils import (
    generate_system_data_region)

from data_specification.enums.data_type import DataType

from spinn_front_end_common.utilities import exceptions
import logging

logger = logging.getLogger(__name__)


class LatticeBasicCell(
    SimulatorVertex, MachineDataSpecableVertex,
    AbstractReceiveBuffersToHost,
    AbstractProvidesNKeysForPartition
):
    """ Cell which represents a cell within the 2d fabric
    """

    PARTITION_ID = "STATE"

    TRANSMISSION_DATA_SIZE = 2 * BYTES_PER_WORD  # has key and key
    STATE_DATA_SIZE = BYTES_PER_WORD  # 1 or 2 based off dead or alive
    # alive states, dead states
    NEIGHBOUR_INITIAL_STATES_SIZE = 2 * BYTES_PER_WORD
    RECORDING_ELEMENT_SIZE = STATE_DATA_SIZE  # A recording of the state

    POSITION_DATA_SIZE = 2 * BYTES_PER_WORD

    NEIGHBOUR_KEYS_SIZE = 9 * BYTES_PER_WORD  # for 8 directions and a mask

    VELOCITY_SIZE = 2 * BYTES_PER_WORD  # u_x and u_y

    # The order of which directions are writeen to sdram
    ORDER_OF_DIRECTIONS = ["N", "W", "S", "E", "NW", "SW", "SE", "NE"]
    # Regions for populations
    DATA_REGIONS = Enum(
        value="DATA_REGIONS",
        names=[('SYSTEM', 0),
               ('TRANSMISSIONS', 1),
               ('POSITION', 2),
               ('NEIGHBOUR_KEYS', 3),
               ('VELOCITY', 4),
               ('RESULTS', 5)])

    def __init__(self, label, x_position, y_position, u_x, u_y):
        super(LatticeBasicCell, self).__init__(label, "lattice_cell.aplx")
        AbstractProvidesNKeysForPartition.__init__(self)
        # app specific data items
        self._x_position = x_position
        self._y_position = y_position
        # self._n_key = n_key
        # self._s_key = s_key
        # self._w_key = w_key
        # self._e_key = e_key
        # self._nw_key = nw_key
        # self._ne_key = ne_key
        # self._sw_key = sw_key
        # self._se_key = se_key
        self.u_x = u_x
        self.u_y = u_y

        self._loccation_vertices = OrderedDict()
        for direction in self.ORDER_OF_DIRECTIONS:
            self._loccation_vertices[direction] = None

    def set_direction_vertex(self, direction, vertex):
        """
        Add a vertex to the corresponding direction
        """
        self._loccation_vertices[direction] = vertex

    def _write_key_data(self, spec, routing_info, graph):
        spec.switch_write_focus(region=self.DATA_REGIONS.NEIGHBOUR_KEYS.value)
        incoming_edges = graph.get_edges_ending_at_vertex_with_partition_name(self, self.PARTITION_ID)

        # verify the number of edges
        if len(incoming_edges) != 8:
            print(incoming_edges)
            raise exceptions.ConfigurationException("Should only have 8 edges")

        # get incoming edges
        for direction in self._loccation_vertices:
            key = routing_info.get_routing_info_from_pre_vertex(
                self._loccation_vertices[direction],
                self.PARTITION_ID).keys_and_masks[0].key
            if key is not None:
                spec.write_value(data=key)
            else:
                logger.warning("This lattice miss a edge from direction {}".format(direction))
                spec.write_value(data_type=DataType.INT32, data=-1)

        mask = routing_info.get_routing_info_from_pre_vertex(self._loccation_vertices["S"],
                                                          self.PARTITION_ID).keys_and_masks[0].mask
        spec.write_value(data=mask, data_type=DataType.UINT32)

    @inject_items({"data_n_time_steps": "DataNTimeSteps"})
    @overrides(
        MachineDataSpecableVertex.generate_machine_data_specification,
        additional_arguments={"data_n_time_steps"})
    def generate_machine_data_specification(
            self, spec, placement, machine_graph, routing_info, iptags,
            reverse_iptags, machine_time_step, time_scale_factor,
            data_n_time_steps):
        # Generate the system data region for simulation .c requirements
        generate_system_data_region(spec, self.DATA_REGIONS.SYSTEM.value,
                                    self, machine_time_step, time_scale_factor)

        # reserve memory regions
        spec.reserve_memory_region(
            region=self.DATA_REGIONS.TRANSMISSIONS.value,
            size=self.TRANSMISSION_DATA_SIZE, label="inputs")

        spec.reserve_memory_region(
            region=self.DATA_REGIONS.POSITION.value,
            size=self.POSITION_DATA_SIZE, label="position"
        )

        spec.reserve_memory_region(
            region=self.DATA_REGIONS.NEIGHBOUR_KEYS.value,
            size=self.NEIGHBOUR_KEYS_SIZE
        )

        spec.reserve_memory_region(
            region=self.DATA_REGIONS.VELOCITY.value,
            size=self.VELOCITY_SIZE
        )

        spec.reserve_memory_region(
            region=self.DATA_REGIONS.RESULTS.value,
            size=recording_utilities.get_recording_header_size(1))

        # get recorded buffered regions sorted
        spec.switch_write_focus(self.DATA_REGIONS.RESULTS.value)
        spec.write_array(recording_utilities.get_recording_header_array(
            [self.RECORDING_ELEMENT_SIZE * data_n_time_steps]))

        # check got right number of keys and edges going into me
        partitions = \
            machine_graph.get_outgoing_edge_partitions_starting_at_vertex(self)
        if not is_single(partitions):
            raise ConfigurationException(
                "Can only handle one type of partition.")

        # check for duplicates
        edges = list(machine_graph.get_edges_ending_at_vertex(self))
        if len(edges) != 8:
            raise ConfigurationException(
                "I've not got the right number of connections. I have {} "
                "instead of 8".format(
                    len(machine_graph.get_edges_ending_at_vertex(self))))

        for edge in edges:
            if edge.pre_vertex == self:
                raise ConfigurationException(
                    "I'm connected to myself, this is deemed an error"
                    " please fix.")

        # write key needed to transmit with
        key = routing_info.get_first_key_from_pre_vertex(
            self, self.PARTITION_ID)

        spec.switch_write_focus(
            region=self.DATA_REGIONS.TRANSMISSIONS.value)
        spec.write_value(0 if key is None else 1)
        spec.write_value(0 if key is None else key)

        # write POSITION data
        spec.switch_write_focus(
            region=self.DATA_REGIONS.POSITION.value
        )
        spec.write_value(int(self._x_position))
        spec.write_value(int(self._y_position))

        # write neighbour keys
        # diraction = ["me", "n", "w", "s", "e", "nw", "sw", "se", "ne"]
        # spec.switch_write_focus(
        #     region=self.DATA_REGIONS.NEIGHBOUR_KEYS.value
        # )
        # spec.write_value(self._n_key)
        # spec.write_value(self._w_key)
        # spec.write_value(self._s_key)
        # spec.write_value(self._e_key)
        # spec.write_value(self._nw_key)
        # spec.write_value(self._sw_key)
        # spec.write_value(self._se_key)
        # spec.write_value(self._ne_key)
        
        #write the neighbour keys and masks
        self._write_key_data(spec, routing_info, machine_graph)
        
        
        spec.switch_write_focus(region=self.DATA_REGIONS.VELOCITY.value)
        spec.write_value(self.u_x, data_type=DataType.FLOAT_32)
        spec.write_value(self.u_y, data_type=DataType.FLOAT_32)

        # End-of-Spec:
        spec.end_specification()

    def get_data(self, buffer_manager, placement):
        # for buffering output info is taken form the buffer manager
        # get raw data, convert to list of booleans
        raw_data, data_missing = buffer_manager.get_data_by_placement(
            placement, 0)

        # do check for missing data
        if data_missing:
            print("missing_data from ({}, {}, {}); ".format(
                placement.x, placement.y, placement.p))

        # return the data, converted to list of booleans
        return [
            element
            for element in struct.unpack(
                "<{}f".format(len(raw_data) // BYTES_PER_WORD), raw_data)]

    @property
    @overrides(MachineVertex.resources_required)
    def resources_required(self):
        fixed_sdram = (SYSTEM_BYTES_REQUIREMENT + self.TRANSMISSION_DATA_SIZE +
                       self.POSITION_DATA_SIZE +
                       self.NEIGHBOUR_KEYS_SIZE +
                       self.VELOCITY_SIZE +
                       recording_utilities.get_recording_header_size(1) +
                       recording_utilities.get_recording_data_constant_size(1))
        per_timestep_sdram = self.RECORDING_ELEMENT_SIZE
        return ResourceContainer(
            sdram=VariableSDRAM(fixed_sdram, per_timestep_sdram))

    @property
    def x_position(self):
        return self._x_position

    @overrides(AbstractProvidesNKeysForPartition.get_n_keys_for_partition)
    def get_n_keys_for_partition(self, partition, graph_mapper):
        return 8 # for its 8 neighbours to send

    @property
    def y_position(self):
        return self._y_position

    def __repr__(self):
        return self.label

    @overrides(AbstractReceiveBuffersToHost.get_recorded_region_ids)
    def get_recorded_region_ids(self):
        return [0]

    @overrides(AbstractReceiveBuffersToHost.get_recording_region_base_address)
    def get_recording_region_base_address(self, txrx, placement):
        return locate_memory_region_for_placement(
            placement, self.DATA_REGIONS.RESULTS.value, txrx)


In [3]:
from pacman.model.graphs.machine import MachineEdge


class LatticeEdge(MachineEdge):
    """
    Used for conjunction with a lattice
    """

    def __init__(self, pre_vertex, post_vertex, compass, label=None):
        MachineEdge.__init__(
            self, pre_vertex, post_vertex, label=label)
        self._compass = compass

    @property
    def compass(self):
        return self._compass

    def __str__(self):
        return self.__repr__()

    def __repr__(self):
        return "LatticeEdge: {}:{}".format(self._compass, self._label)

In [4]:
# Copyright (c) 2017-2019 The University of Manchester
#
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with this program.  If not, see <http://www.gnu.org/licenses/>.
import math
import os
# from pacman.model.graphs.machine import MachineEdge

import spinnaker_graph_front_end as front_end


runtime = 5000
time_step = 100000
MAX_X_SIZE_OF_FABRIC = 20
MAX_Y_SIZE_OF_FABRIC = 20
n_chips = (MAX_X_SIZE_OF_FABRIC * MAX_Y_SIZE_OF_FABRIC) // 15

ex = [0, 1, 0, -1, 0, 1, -1, -1, 1]
ey = [0, 0, 1, 0, -1, 1, 1, -1, -1]


def generatePositionMap(x_dim, y_dim):
    positionMap = {}
    #   diraction = ["me", "n", "w", "s", "e", "nw", "sw", "se", "ne"]
    for i in range(9):
        x_temp = int(x_dim - ex[i]) % MAX_X_SIZE_OF_FABRIC
        y_temp = int(y_dim - ey[i]) % MAX_Y_SIZE_OF_FABRIC
        positionMap[i] = (x_temp, y_temp)
    return positionMap


def generateKeys(position):
    return position[0] * MAX_X_SIZE_OF_FABRIC + position[1]


def initVelocity(x_pos, y_pos):
    U_0 = 0.01
    K = 30.0
    delta = 0.05
    x_temp = 1.0 * (x_pos) / MAX_X_SIZE_OF_FABRIC
    y_temp = 1.0 * (y_pos) / MAX_Y_SIZE_OF_FABRIC
    if y_temp <= 0.5:
        u_x = U_0 * math.tanh(K * (y_temp - 0.25))
    else:
        u_x = U_0 * math.tanh(K * (0.75 - y_temp))
    #     print("the u_x of ({}, {}) is {}".format(x_pos, y_pos, u_x))
    u_y = U_0 * delta * math.sin(2 * math.pi * (x_temp + 0.25))
    #     print("the u_y of ({}, {}) is {}".format(x_pos, y_pos, u_y))
    return u_x, u_y


# set up the front end and ask for the detected machines dimensions
front_end.setup(
    n_chips_required=n_chips, model_binary_folder=os.path.dirname(os.path.abspath("__file__")), machine_time_step=time_step)

# figure out if machine can handle simulation
cores = front_end.get_number_of_available_cores_on_machine()
# print(cores)
if cores <= (MAX_X_SIZE_OF_FABRIC * MAX_Y_SIZE_OF_FABRIC):
    raise KeyError("Don't have enough cores to run simulation")

# contain the vertices for the connection aspect
vertices = [
    [None for _ in range(MAX_X_SIZE_OF_FABRIC)]
    for _ in range(MAX_Y_SIZE_OF_FABRIC)]

# build vertices
for x in range(0, MAX_X_SIZE_OF_FABRIC):
    for y in range(0, MAX_Y_SIZE_OF_FABRIC):
        u_x, u_y = initVelocity(x, y)
#         keymap = generatePositionMap(x, y)
        vert = LatticeBasicCell(
            "cell{}".format((x * MAX_X_SIZE_OF_FABRIC) + y),
            x, y, u_x, u_y)
        vertices[x][y] = vert
        front_end.add_machine_vertex_instance(vert)

# verify the initial state
output = ""
for x in range(0, MAX_X_SIZE_OF_FABRIC):
    for y in range(0, MAX_Y_SIZE_OF_FABRIC):
        output += "({0}, {1}) ".format(vertices[x][y].x_position, vertices[x][y].y_position)
    output += "\n"
print(output)
print("\n\n")

# build edges
for x in range(0, MAX_X_SIZE_OF_FABRIC):
    for y in range(0, MAX_Y_SIZE_OF_FABRIC):
#         keymap = generatePositionMap(x, y)
        #   diraction = ["me", "n", "w", "s", "e", "nw", "sw", "se", "ne"]
        positions = [
            (x, (y + 1) % MAX_Y_SIZE_OF_FABRIC, "E"),
            ((x + 1) % MAX_X_SIZE_OF_FABRIC,
             (y + 1) % MAX_Y_SIZE_OF_FABRIC, "SE"),
            ((x + 1) % MAX_X_SIZE_OF_FABRIC, y, "S"),
            ((x + 1) % MAX_X_SIZE_OF_FABRIC,
             (y - 1) % MAX_Y_SIZE_OF_FABRIC, "SW"),
            (x, (y - 1) % MAX_Y_SIZE_OF_FABRIC, "W"),
            ((x - 1) % MAX_X_SIZE_OF_FABRIC,
             (y - 1) % MAX_Y_SIZE_OF_FABRIC, "NW"),
            ((x - 1) % MAX_X_SIZE_OF_FABRIC, y, "N"),
            ((x - 1) % MAX_X_SIZE_OF_FABRIC,
             (y + 1) % MAX_Y_SIZE_OF_FABRIC, "NE")]

         # build edges for each direction for this vertex
        for (dest_x, dest_y, compass) in positions:
            front_end.add_machine_edge_instance(LatticeEdge( vertices[x][y], vertices[dest_x][dest_y],compass, "edge between {} and {}".format(vertices[x][y], vertices[dest_x][dest_y])), LatticeBasicCell.PARTITION_ID)
            vertices[x][y].set_direction_vertex(direction=compass, vertex=vertices[dest_x][dest_y])

# run the simulation
front_end.run(runtime)

# get recorded data
recorded_data = dict()

# if not front_end.use_virtual_machine():
buffer_manager = front_end.buffer_manager()

# get the data per vertex
for x in range(0, MAX_X_SIZE_OF_FABRIC):
    for y in range(0, MAX_Y_SIZE_OF_FABRIC):
        recorded_data[x, y] = vertices[x][y].get_data(
            front_end.buffer_manager(),
            front_end.placements().get_placement_of_vertex(
                vertices[x][y]))

# visualise it in text form (bad but no vis this time)
# for time in range(0, runtime*1000//time_step):
#     print("at time {}".format(time))
#     output = ""
#     for x in range(0, MAX_Y_SIZE_OF_FABRIC):
#         for y in range(0, MAX_Y_SIZE_OF_FABRIC):
#             output += "{} ".format(recorded_data[x, y][time])
#         output += "\n"
#     print(output)
#     print("\n\n")

# clear the machine
front_end.stop()


2020-07-13 15:18:00 INFO: Read cfg files: /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg, /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinnaker_graph_front_end/spiNNakerGraphFrontEnd.cfg, /home/spinnaker/.spiNNakerGraphFrontEnd.cfg
2020-07-13 15:18:00 INFO: Will search these locations for binaries: /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/common_model_binaries : /home/spinnaker
2020-07-13 15:18:00 WARNING: /home/spinnaker/reports has 13 old reports that have not been closed
2020-07-13 15:18:00 WARNING: /home/spinnaker/application_generated_data_files has 110 old reports that have not been closed
2020-07-13 15:18:00 INFO: Setting time scale factor to 2.
2020-07-13 15:18:00 INFO: Setting machine time step to 100000 micro-seconds.
Created spalloc job 5330687
2020-07-13 15:18:00 INFO: Created spalloc job 5330687
Waiting for board power commands to complete.
2020-07-13 15:18:00 INFO: Waiting 

['/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg', '/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinnaker_graph_front_end/spiNNakerGraphFrontEnd.cfg', '/home/spinnaker/.spiNNakerGraphFrontEnd.cfg']


2020-07-13 15:18:09 INFO: Time 0:00:08.940734 taken by SpallocAllocator
2020-07-13 15:18:09 INFO: Creating transceiver for 10.11.193.1
2020-07-13 15:18:09 INFO: Working out if machine is booted
2020-07-13 15:18:13 INFO: Attempting to boot machine
2020-07-13 15:18:18 INFO: Found board with version [Version: SC&MP 3.2.5 at SpiNNaker:0:0:0 (built Thu Aug  1 08:15:06 2019)]
2020-07-13 15:18:18 INFO: Machine communication successful
2020-07-13 15:18:18 INFO: Detected a machine on IP address 10.11.193.1 which has 856 cores and 120.0 links
2020-07-13 15:18:18 INFO: Time 0:00:09.636316 taken by MachineGenerator
Pre allocating resources for Extra Monitor support vertices
|0%                          50%                         100%|
2020-07-13 15:18:18 INFO: Time 0:00:00.022791 taken by PreAllocateResourcesForExtraMonitorSupport
2020-07-13 15:18:18 INFO: Starting execution process
2020-07-13 15:18:18 INFO: Time 0:00:00.005682 taken by NetworkSpecificationReport
Allocating virtual identifiers
|0

(0, 0) (0, 1) (0, 2) (0, 3) (0, 4) (0, 5) (0, 6) (0, 7) (0, 8) (0, 9) (0, 10) (0, 11) (0, 12) (0, 13) (0, 14) (0, 15) (0, 16) (0, 17) (0, 18) (0, 19) 
(1, 0) (1, 1) (1, 2) (1, 3) (1, 4) (1, 5) (1, 6) (1, 7) (1, 8) (1, 9) (1, 10) (1, 11) (1, 12) (1, 13) (1, 14) (1, 15) (1, 16) (1, 17) (1, 18) (1, 19) 
(2, 0) (2, 1) (2, 2) (2, 3) (2, 4) (2, 5) (2, 6) (2, 7) (2, 8) (2, 9) (2, 10) (2, 11) (2, 12) (2, 13) (2, 14) (2, 15) (2, 16) (2, 17) (2, 18) (2, 19) 
(3, 0) (3, 1) (3, 2) (3, 3) (3, 4) (3, 5) (3, 6) (3, 7) (3, 8) (3, 9) (3, 10) (3, 11) (3, 12) (3, 13) (3, 14) (3, 15) (3, 16) (3, 17) (3, 18) (3, 19) 
(4, 0) (4, 1) (4, 2) (4, 3) (4, 4) (4, 5) (4, 6) (4, 7) (4, 8) (4, 9) (4, 10) (4, 11) (4, 12) (4, 13) (4, 14) (4, 15) (4, 16) (4, 17) (4, 18) (4, 19) 
(5, 0) (5, 1) (5, 2) (5, 3) (5, 4) (5, 5) (5, 6) (5, 7) (5, 8) (5, 9) (5, 10) (5, 11) (5, 12) (5, 13) (5, 14) (5, 15) (5, 16) (5, 17) (5, 18) (5, 19) 
(6, 0) (6, 1) (6, 2) (6, 3) (6, 4) (6, 5) (6, 6) (6, 7) (6, 8) (6, 9) (6, 10) (6, 11) (6, 12) 

Writing the board chip report
|0%                          50%                         100%|
2020-07-13 15:18:19 INFO: Time 0:00:00.014657 taken by BoardChipReport
Placing graph vertices
|0%                          50%                         100%|
2020-07-13 15:18:19 INFO: Time 0:00:00.116340 taken by RadialPlacer
Inserting edges between vertices which require FR speed up functionality.
|0%                          50%                         100%|
2020-07-13 15:18:19 INFO: Time 0:00:00.096409 taken by InsertEdgesToExtraMonitorFunctionality
Generating routing tables for data in system processes
|0%                          50%                         100%|
2020-07-13 15:18:19 INFO: Time 0:00:00.008757 taken by DataInMulticastRoutingGenerator
Generating fixed router routes
|0%                          50%                         100%|
2020-07-13 15:18:19 INFO: Time 0:00:00.007559 taken by FixedRouteRouter
Routing
|0%                          50%                         100%|

2020-07-

Getting provenance data from machine graph
|0%                          50%                         100%|
2020-07-13 15:18:40 INFO: Time 0:00:00.168481 taken by GraphProvenanceGatherer
2020-07-13 15:18:40 INFO: Time 0:00:00.000285 taken by DatabaseInterface
2020-07-13 15:18:40 INFO: ** Notifying external sources that the database is ready for reading **
2020-07-13 15:18:40 INFO: Time 0:00:00.000839 taken by NotificationProtocol
2020-07-13 15:18:40 INFO: *** Running simulation... *** 
Loading buffers
|0%                          50%                         100%|
2020-07-13 15:18:40 INFO: ** Awaiting for a response from an external source to state its ready for the simulation to start **
2020-07-13 15:18:40 INFO: ** Sending start / resume message to external sources to state the simulation has started or resumed. **
2020-07-13 15:18:40 INFO: ** Awaiting for a response from an external source to state its ready for the simulation to start **
2020-07-13 15:18:40 INFO: Application started; 

In [14]:
for time in range(0, runtime*1000//time_step):
    print("at time {}".format(time))
    output = ""
    for x in range(0, MAX_Y_SIZE_OF_FABRIC):
        for y in range(0, MAX_Y_SIZE_OF_FABRIC):
            output += "{} ".format(recorded_data[x, y][time])
        output += "\n"
    print(output)
    print("\n\n")

In [6]:
len(recorded_data[1,1])

50

In [20]:
# visualise it in text form (bad but no vis this time)
u_x_output = collections.defaultdict(dict)
u_y_output = collections.defaultdict(dict)
for i in range(0, MAX_X_SIZE_OF_FABRIC):
    for j in range(0, MAX_Y_SIZE_OF_FABRIC):
        u_x_output[i][j] = recorded_data[i, j][998]
        u_y_output[i][j] = recorded_data[i, j][999]
u_x_output

defaultdict(dict,
            {0: {0: -0.009396855719387531,
              1: -0.010218574665486813,
              2: -0.006607631221413612,
              3: 0.005369159393012524,
              4: 0.01079240720719099,
              5: 0.009396894834935665,
              6: 0.010218577459454536,
              7: 0.006607951130717993,
              8: -0.0053691319189965725,
              9: -0.010792539454996586},
             1: {0: -0.0093629015609622,
              1: -0.010568411089479923,
              2: -0.006551017984747887,
              3: 0.0055407509207725525,
              4: 0.011004633270204067,
              5: 0.00943238940089941,
              6: 0.010002228431403637,
              7: 0.006457724142819643,
              8: -0.0054624550975859165,
              9: -0.010495664551854134},
             2: {0: -0.0093772541731596,
              1: -0.010892486199736595,
              2: -0.006289700511842966,
              3: 0.005901691969484091,
              4: 0.011052

In [18]:
u_y_output

defaultdict(dict,
            {0: {0: 0.00046017352724447846,
              1: 0.0004914472810924053,
              2: 0.000501154107041657,
              3: 0.00042812040192075074,
              4: 0.00044137370423413813,
              5: 0.00046010088408365846,
              6: 0.0004914304590784013,
              7: 0.0005011206376366317,
              8: 0.000428129656938836,
              9: 0.00044139413512311876},
             1: {0: 0.00034707237500697374,
              1: 0.00040169956628233194,
              2: 0.000425032339990139,
              3: 0.0003268826985731721,
              4: 0.0003519370511639863,
              5: 0.00039747735718265176,
              6: 0.00040300769614987075,
              7: 0.0003996076702605933,
              8: 0.00035201915306970477,
              9: 0.00035270547959953547},
             2: {0: 0.00010146568092750385,
              1: 0.00016870888066478074,
              2: 0.00020073851919732988,
              3: 8.65916590555571e-05,
 

In [ ]:
voriticity = collections.defaultdict(dict)

for i in range(0, MAX_X_SIZE_OF_FABRIC):
    for j in range(0, MAX_Y_SIZE_OF_FABRIC): 
        voriticity[i][j] = 0.5 * (u_y_output[(i + 1) % MAX_X_SIZE_OF_FABRIC][j] - u_y_output[(i - 1)%MAX_X_SIZE_OF_FABRIC][j]) + 0.5 * (u_x_output[i][(j + 1)%MAX_Y_SIZE_OF_FABRIC] - u_x_output[i][(j - 1) % MAX_Y_SIZE_OF_FABRIC])

voriticity

In [8]:
# Check the whole mass, if it keeo the same, then it is correct
for time in range(0,50):
    wholemass = 0
    for i in range(0, MAX_X_SIZE_OF_FABRIC):
        for j in range(0, MAX_Y_SIZE_OF_FABRIC): 
            wholemass+=recorded_data[i,j][time]
    print(wholemass)

16385.894458770752
16386.10289502144
16386.12949717045
16386.129251360893
16386.13218474388
16386.129850625992
16386.131890177727
16386.130145192146
16386.131560087204
16386.13037610054
16386.13133609295
16386.130592226982
16386.131173729897
16386.130736470222
16386.13106930256
16386.130801558495
16386.13099706173
16386.13082897663
16386.130931138992
16386.13084566593
16386.130893707275
16386.130850195885
16386.130888700485
16386.13085412979
16386.130864024162
16386.130844950676
16386.130851387978
16386.130819678307
16386.130818367004
16386.13080382347
16386.13082611561
16386.130828380585
16386.130821704865
16386.13081085682
16386.130801439285
16386.130797624588
16386.1307772398
16386.13076066971
16386.130743145943
16386.13073539734
16386.130707263947
16386.130701184273
16386.13068962097
16386.13069176674
16386.130684256554
16386.130683898926
16386.130676150322
16386.13065493107
16386.13066279888
16386.130648612976


In [7]:
# Check hte momentum. if all 0 then it is correct
for time in range(50):
    wholeMomentum = 0
    for i in range(0, MAX_X_SIZE_OF_FABRIC):
        for j in range(0, MAX_Y_SIZE_OF_FABRIC): 
            wholeMomentum+=recorded_data[i,j][time]
    print(wholeMomentum)

6.51925802230835e-08
9.31322801989154e-07
1.426786129243851e-06
1.3315584652673351e-06
9.806826675173852e-07
1.0684595963539323e-06
8.31438001114293e-07
8.535581592639119e-07
1.3420365121419309e-06
1.5620607882738113e-06
1.6086289633676643e-06
1.0952326192636974e-06
8.40511802380206e-08
1.702223926258739e-06
1.287542090722127e-07
-9.534473974781577e-07
-1.2684581633948255e-06
-1.1054780770791695e-06
-1.5702144082752056e-06
-4.258463377482258e-07
-2.1233813640719745e-07
2.8172337351861643e-07
6.67758740746649e-07
8.991896720544901e-07
1.3252669077701285e-06
8.039613703658688e-07
9.725320069264853e-07
9.428924840904074e-08
6.046639100532047e-07
1.5969920923453174e-06
2.3741769155094516e-06
2.3325221718550893e-06
2.251924115626025e-06
2.52110419296514e-06
1.7159754861495458e-06
1.6710209820303135e-06
2.4633477586633035e-06
1.4421489158422673e-06
1.7513557395432144e-06
1.3927956388215534e-06
1.3264225344755687e-06
2.047054746157073e-06
1.1902175174327567e-06
3.9021392694849055e-07
2.642536